In [1]:
import pandas as pd
import numpy as np
from modules import *

In [2]:
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk import word_tokenize
from konlpy.tag import Okt

In [27]:
from nltk.util import everygrams

#원하는 태그 추출 & 불용어 제거
okt = Okt()

#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",
                 '에서','고','이다','젛','뇨','껀데','뭘'
                 ,'최고','느리다','마을','최고','항상','포장','재다','말다','목','넘김','만족하다','정말','역시','아주','감사하다','오다','자주','요','늘','넘다','않다','더','병','다','숙성','나다','술','드리다','마트','꼼꼼하다','잇다'
                 ,'복','순도','용','비싸다','가격','많다','강하다','보내다','추천','보고','쏘다','느낌','사다','처음','많이','톡','생각','엄청','날','아니다']) #불용어 추가

stopwords=set(stopwords) # 중복제거    

# 형태소 분석 함수 만들기
def okt_pos_tagging2(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    # n_gram 만들기
    egram = list(everygrams(words, min_len=1, max_len=1))
    egram_token = [' '.join(grams) for grams in egram]

    return egram_token

def okt_pos_tagging(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    return words


In [28]:
# 불러올 csv 이름 리스트
# 전체 리뷰를 불러온후 tf-idf 분석
name_list =["술 리뷰(곰표막걸리)","술 리뷰(복순도가)","느린마을 막걸리","술 리뷰(죽향도가)"]

In [29]:
all_data = pd.DataFrame()

for name in name_list:
    data = pd.read_csv(f"{name}.csv", encoding='utf-8-sig')
    # 전처리
    data.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거
    try:
        data.columns = ['date', 'star','doc']
    except:
        data.columns = ['date', 'star','doc','name']
    data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
    data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
    data.doc = data.doc.str.replace("럽고목","럽고 목")
    data.doc = data.doc.str.replace("부 드럽","부드럽")
    data.dropna(inplace=True)
    data.reset_index(drop=True, inplace=True)
    # globals()["감홍로 40도"] 등으로 데이터 프레임 저장됨
    globals()[f"{name}"] = data
    all_data = pd.concat([all_data,data],axis=0)


C:\Users\user\AppData\Local\Temp\ipykernel_15056\3516534953.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_15056\3516534953.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_15056\3516534953.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_15056\3516534953.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_15056\3516534953.py:11: FutureWarning

In [30]:
all_data

,date,star,doc,name
0,2022-11-22,5,표문 막걸리는 제가 제일 좋아하는 막걸리예요표문막걸리 런칭 첫 라이브 에서부터 편의...,NaN
1,2022-11-22,5,와라이브 이벤트에 후디 당첨되었어요와 진짜 당첨될줄 몰랐는데 너무 기분이 좋네요후디...,NaN
2,2022-11-26,5,한강주조를 처음 알게된건 인사동 어느 고기집이었는데 부모님과 함께 간단히 마시려고 ...,NaN
3,2022-11-28,5,축구경기 보며 마셨어요부드럽고 은은한 단맛이 너무 좋아요계속 생각나는 맛이네요 라이...,NaN
4,2022-11-24,5,가격이 많이 내렸네여곰표 막걸리 중독성있습니다쏘는맛없지만 부드럽고 맛있어요,NaN
...,...,...,...,...
5674,2020-04-03,5,다양한술을 집에서 편하게 주문가능하기에 자주 주문합니다이번에도 즐겁게 마시겠읍니다,NaN
5675,2020-04-03,5,빠른 배송 감사합니당,NaN
5676,2020-03-31,5,단맛은 있는데 꿀 단맛이어서 나쁘지 않아요 기분 나쁘지 않은 단맛이에요 인공감미료가...,NaN
5677,2020-03-26,5,맛이 달고 청량감이 좋아요,NaN


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=okt_pos_tagging)
tfidf_fit = tfidf.fit(all_data.doc)


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [32]:
# 특징을 추출할 문서의 tf-idf를 구함
# "술 리뷰(곰표막걸리)","술 리뷰(복순도가)","느린마을 막걸리"
bow_rep_tfidf = tfidf_fit.transform(globals()["술 리뷰(죽향도가)"].doc)
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names(),
    'tf-idf': bow_rep_tfidf.sum(axis=0).flat,
    'idf': tfidf.idf_
})



c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [33]:
# idf 가 높을수록 잘 안나오는 단어
# tf-idf 가 높을수록 중요한 단어
# 두값을 적절히 조정해서 특징 추출
word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.9][word_count['tf-idf']>30]

C:\Users\user\AppData\Local\Temp\ipykernel_15056\3393798343.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.9][word_count['tf-idf']>30]


,단어,tf-idf,idf
5461,샴페인,245.691394,5.501938
4768,부모님,241.097866,5.122699
113,가족,187.844522,5.283560
9265,지인,183.939698,5.068860
9953,친구,178.239521,5.133604
2169,달달,170.036841,4.140109
3750,명절,164.993022,5.783826
1303,깔끔하다,139.744876,4.423065
7357,와인,135.230157,5.828947
6160,시큼하다,128.429487,5.853419


In [ ]:
def generate_output(self):
        """
        Final output--to be displayed on the app
        Columns: documents, top topic, n top words for topic, 2 corresponding sentences per word?
        """
        file_to_tokens = self._get_normalized_corpus(self.files)
        all_lemmas = self._get_all_unique_lemmas(file_to_tokens)

        doc_topic = self.lda_model.doc_topic_  # document-topic distributions
        topic_word = self.lda_model.topic_word_  # topic-word distributions

        # Map lemmas to tokens
        lemma_to_tokens = self._get_lemma_to_tokens(file_to_tokens)

        # Map topic indices to lemmas
        topic_lemmas_map = {}
        for i, topic_dist in enumerate(topic_word):
            top_lemmas = list(np.array(all_lemmas)[np.argsort(topic_dist)][
                : -self.n_top_words : -1
            ])  # list of top lemmas
            top_lemmas_to_tokens = {}
            for lemma in top_lemmas:
                top_tokens = lemma_to_tokens[lemma]  # Grab all tokens corresponding to that lemma
                top_lemmas_to_tokens[lemma] = top_tokens
            topic_lemmas_map[i] = top_lemmas_to_tokens

        # Final output: a map of doc_names to topic number, lemmas assoc. with that topic
        # and the tokens associated with those words
        doc_topic_words = {}
        for i in range(len(self.files)):
            top_topic = doc_topic[i].argmax()
            current_file = self.files[i]
            doc_topic_words[self.files[i]] = {top_topic: topic_lemmas_map[top_topic]}

        self._doc_topic_words = doc_topic_words